In [ ]:
!pip install tensorly


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from queue import PriorityQueue
import tensorly
from tensorly.decomposition import non_negative_parafac
import math
from tensorly.metrics import regression
import random




# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
def selectRandomSamples(A1slice,A2slice,A3slice,s1,s2,s3):

    #moi along I:
    pqI = PriorityQueue()
    sum_weight = 0
    for i in range(0,A1slice.shape[0]):
        sum_weight = 0
        for j in range(0,A1slice.shape[1]):
            for k in range(0,A1slice.shape[2]):
                #print("c:",A1slice[i][j][k])
                sum_weight += A1slice[i][j][k]
        pqI.put((-sum_weight,i))



    selectedI = np.zeros([s1,A1slice.shape[1],A1slice.shape[2]])
    selectedindxI = np.zeros(s1)
    i = 0
    print(tf.shape(A)[0])
    while not pqI.empty():
        next_item = pqI.get()
        print(next_item)
        if i < s1:
            selectedI[i] = A1slice[next_item[1]]
            selectedindxI[i] = next_item[1]
        i += 1




    print("SI:",selectedI)
    print("jkljk")

    #moi along J:
    print(A)
    pqJ = PriorityQueue()
    sum_weight = 0
    print("A2slice:",A2slice)
    for i in range(0,A2slice.shape[0]):
        sum_weight = 0
        for j in range(0,A2slice.shape[1]):
            for k in range(0,A2slice.shape[2]):
                #print("c:",A1slice[i][j][k])
                sum_weight += A2slice[i][j][k]
        pqJ.put((-sum_weight,i))

    selectedJ = np.zeros([s2,A2slice.shape[1],A2slice.shape[2]])
    selectedindxJ = np.zeros(s2)
    i = 0

    while not pqJ.empty():
        next_item = pqJ.get()
        print("nxt:",next_item)
        if i < s2:
            print(i)
            selectedJ[i] = A2slice[next_item[1]]
            selectedindxJ[i] = next_item[1]
        i += 1

    print("j:",selectedJ)

    #moi along K:
    pqK = PriorityQueue()
    sum_weight = 0
    print("A3slice:",A3slice)
    for i in range(0,A3slice.shape[0]):
        sum_weight = 0
        for j in range(0,A3slice.shape[1]):
            for k in range(0,A3slice.shape[2]):
                #print("c:",A1slice[i][j][k])
                sum_weight += A3slice[i][j][k]
        pqK.put((-sum_weight,i))

    selectedK = np.zeros([s3,A3slice.shape[1],A3slice.shape[2]])
    selectedindxK = np.zeros(s3)
    print(selectedK.shape)
    i = 0

    while not pqK.empty():
        next_item = pqK.get()
        print(next_item)
        if i < s3:
            selectedK[i] = A3slice[next_item[1]]
            selectedindxK[i] = next_item[1]
        i += 1

    print(selectedK)
    return selectedI,selectedJ,selectedK,selectedindxI,selectedindxJ,selectedindxK





In [ ]:
def buildXs(selectedI,selectedJ,selectedK,F):
    if F == None:
        Xs = np.zeros([(selectedI.shape[0]+selectedJ.shape[0]+selectedK.shape[0]),max(selectedI.shape[1],selectedJ.shape[1],selectedK.shape[1]),max(selectedI.shape[2],selectedJ.shape[2],selectedK.shape[2])])
    else:
        fixed1 = F[0]
        fixed2 = F[1]
        fixed3 = F[2]
        Xs = np.zeros([(selectedI.shape[0]+selectedJ.shape[0]+selectedK.shape[0]+fixed1.shape[0]+fixed2.shape[0]+fixed3.shape[0]),max(selectedI.shape[1],selectedJ.shape[1],selectedK.shape[1],fixed1.shape[1],fixed2.shape[1],fixed3.shape[1]),max(selectedI.shape[2],selectedJ.shape[2],selectedK.shape[2],fixed1.shape[2],fixed2.shape[2],fixed3.shape[2])])
    print(Xs.shape)
    l=0
    m=0
    n=0

    for i in range(0,selectedI.shape[0]):
        for j in range(0,selectedI.shape[1]):
            for k in range(0,selectedI.shape[2]):
                Xs[i][j][k] = selectedI[i][j][k]
                l=i
                m=j
                n=k
                
    # Add new incoming slices
    if F != None:
        for i in range(0,fixed1.shape[0]):
            l+=1
            for j in range(0,fixed1.shape[1]):
                for k in range(0,fixed1.shape[2]):
                    Xs[l][j][k] = fixed1[i][j][k]

    for i in range(0,selectedJ.shape[0]):
        l+=1
        for j in range(0,selectedJ.shape[1]):
            for k in range(0,selectedJ.shape[2]):
                Xs[l][j][k] = selectedJ[i][j][k]
    
    if F != None:
        for i in range(0,fixed2.shape[0]):
            l+=1
            for j in range(0,fixed2.shape[1]):
                for k in range(0,fixed2.shape[2]):
                    Xs[l][j][k] = fixed2[i][j][k]

    for i in range(0,selectedK.shape[0]):
        l+=1
        for j in range(0,selectedK.shape[1]):
            for k in range(0,selectedK.shape[2]):
                Xs[l][j][k] = selectedK[i][j][k]
    if F != None:
        for i in range(0,fixed3.shape[0]):
            l+=1
            for j in range(0,fixed3.shape[1]):
                for k in range(0,fixed3.shape[2]):
                    Xs[l][j][k] = fixed3[i][j][k]


    print(Xs)
    return Xs


In [ ]:
def summary(A,s,fixed = None):
    #determine sample sizes
    I = A.shape[0]
    J = A.shape[1]
    K = A.shape[2]
    s1 = math.ceil(I/s[0])
    s2 = math.ceil(J/s[1])
    s3 = math.ceil(K/s[2])
    print("S:",s1,s2,s3)
    
    #Slice input matrix A along all dimensions
    A1slice = np.array(tf.slice(A, [0, 0, 0], [tf.shape(A)[0],tf.shape(A)[1] , tf.shape(A)[2]]))
    print("A1slices shape:",A1slice.shape)
    print("------------------------------------")
    print("A1slice:")
    print(A1slice)
    A2slice = np.zeros([A.shape[1],A.shape[0],A.shape[2]])
    for i in range(0,tf.shape(A)[1]):
        col = tf.slice(A, [0, i, 0], [tf.shape(A)[0],1 , tf.shape(A)[2]])
        A2slice[i] = col[:,0]
    print("------------------------------------")
    print("A2slice:")
    print(A2slice)
    A3slice = np.zeros([A.shape[2],A.shape[0],A.shape[1]])
    for i in range(0,tf.shape(A)[2]):
        col = tf.slice(A, [0, 0, i], [tf.shape(A)[0],tf.shape(A)[1] , 1])
        A3slice[i] = col[:,:,0]
    print("------------------------------------")
    print("A3slice:")
    print(A3slice)
    
    #Choose random samples along modes 1,2,3
    selectedI,selectedJ,selectedK,selectedindxI,selectedindxJ,selectedindxK = selectRandomSamples(A1slice,A2slice,A3slice,s1,s2,s3)
    print("------------------------------------")
    print("selectedI:")
    print(selectedI)
    print("------------------------------------")
    print("selectedJ:")
    print(selectedJ)
    print("------------------------------------")
    print("selectedK:")
    print(selectedK)
    #Combine samples
    if(fixed == None):
        Xs = buildXs(selectedI,selectedJ,selectedK,F = None)
    else:
        Xs = buildXs(selectedI,selectedJ,selectedK,F = fixed)
    print("------------------------------------")
    print("XS:")
    print(Xs)
    
    return Xs,selectedI,selectedJ,selectedK,selectedindxI,selectedindxJ,selectedindxK
    

In [ ]:
def buildXnew(Xold,F,mode1,mode2,mode3):
    #Xslices tensor shape:
    m = mode1.shape[0] 
    n = mode2.shape[0] 
    p = mode3.shape[0]
    
    #Build Xslices tensor with mode1,2 and 3 selected slices from previoud tensor
    Xslices = np.zeros([m,n,p])
    """for i in range(0,m):
        for j in range(0,n):
            for k in range(0,p):
                Xslices[i][j][k] = Xold[int(mode1[i])][int(mode2[j])][int(mode3[k])]"""
    
    Xslices = buildXs(mode1,mode1,mode3,F = None)
                
    
    print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
    print("Xslices:")
    print(Xslices)
    
    
    #Merge Xslices and new incoming slices
    m = max(Xslices.shape[0],F.shape[0])
    n = max(Xslices.shape[1],F.shape[1])
    p = Xslices.shape[2] + F.shape[2]
    new = 0
    Xnew = np.zeros([m,n,p])
    print("Shapes:")
    print("m:",str(m),"n:",str(n),"p:",str(p))
    print(F.shape[0],F.shape[1],F.shape[2])
    print("Xslices Shape:"+str(Xslices.shape))
    for i in range(0,m):
        for j in range(0,n):
            for k in range(0,p):
                if k < Xslices.shape[2]:
                    Xnew[i][j][k] = Xslices[i][j][k]
                elif k >= Xslices.shape[2] and new < F.shape[2] and i < F.shape[0] and j < F.shape[1]:
                    Xnew[i][j][k] = F[i][j][new]
                    new+=1
            new = 0
                
        
   
    
                
                
    return Xnew
    
                

In [ ]:
def merge(As,Bs,Cs,r, rank,selectedI,selectedJ, selectedK ,Aold=None,Bold=None,Cold=None):
    usedA =[]
    if r ==0:
        A = As
        B = Bs
        C = Cs
    else:
        vA=np.zeros([Aold.shape[0],1])
        vB=np.zeros([Bold.shape[0],1])
        vC=np.zeros([Cold.shape[0],1])
        
        for i in range(0,rank):
            for j in range(0,rank):
                print(np.dot(Aold[:,i] , As[:,j]))
                
                vA[j] = np.dot(Aold[:,i] , As[:,j])
                vB[j] = np.dot(Bold[:,i] , Bs[:,j])
                vC[j] = np.dot(Cold[:,i] , Cs[:,j])
                
            idx = np.argsort(vA)
            idx = idx[::-1]
            
            ii=0
            while(idx[ii] in usedA):
                ii= ii+1
            
            Id = idx[ii]
            usedA.append(Id)
            print(usedA)
            
            for k in range(0,As.shape[0]):
                if As[k][Id] == 0:
                    As[k][Id] = As[k][Id] + Aold[k][i]
                    
            for k in range(0,Bs.shape[0]):
                if Bs[k][Id] == 0:
                    Bs[k][Id] = Bs[k][Id] + Bold[k][i]
                    
            for k in range(0,Cs.shape[0]):
                if Cs[k][Id] == 0:
                    Cs[k][Id] = Cs[k][Id] + Cold[k][i]
        
        A = As
        B = Bs
        C = Cs
    
    return A, B, C
        
    
        
    

In [ ]:
#Load dataset 
userdata = pd.read_csv('../input/credit-card-approval-prediction/application_record.csv')
recorddata = pd.read_csv('../input/credit-card-approval-prediction/credit_record.csv')
print(recorddata)

In [ ]:
#Process dataset
userdata['STATUS']= recorddata['STATUS']
userdata = userdata.loc[0:330,:] 
print(userdata.shape)
row= userdata.shape[0]
print(row)
a = [1,2,3]

#userdata['creditType'] = [ "Bad" if (x == '1' or x == '2' or x == '3' or x == '4' or x == '5') else "Good" for x in userdata.STATUS]
userdata['Status'] = [ 1 if (x == '0') else 1 for x in userdata.STATUS]

for i,j in userdata.iterrows():
    if j.STATUS == '1':
        j.Status = 1 
    elif j.STATUS == '2':
        j.Status = 2 
    elif j.STATUS == '3':
        j.Status = 3 
    elif j.STATUS == '4':
        j.Status = 4 
    elif j.STATUS == '5':
        j.Status = 5 
    elif j.STATUS == 'X':
        j.Status = 6
    elif j.STATUS == 'C':
        j.Status = 7
    elif j.STATUS == '0':
        j.Status = 1
        
#randomly placing numbers in status
for i in range(0,330):
    x = random.randint(2,5)
    rowno = random.randint(0,330)
    userdata.iloc[rowno,19] = x
    


headers=list(userdata.columns)
print(headers)
a=list(userdata.loc[0])
print(a)

userdata.to_csv('csv_to_submit.csv', index = False)
df=userdata[['AMT_INCOME_TOTAL','FLAG_OWN_CAR','DAYS_EMPLOYED','CNT_FAM_MEMBERS']].copy()



df['DAYS_EMPLOYED']=df['DAYS_EMPLOYED'].astype('category').cat.codes
df['FLAG_OWN_CAR']=df['FLAG_OWN_CAR'].astype('category').cat.codes

df.corr(method ='kendall') 

In [ ]:
#Apply PCA
from sklearn.decomposition import PCA 
pca = PCA(n_components=1)
principalComponents = pca.fit_transform(df)
principalDf = pd.DataFrame(principalComponents, columns = ['Economic_Indicator'])
principalDf['ID']=userdata['ID']
principalDf['Status']= userdata['Status']
principalDf['OCCUPATION_TYPE']= userdata['OCCUPATION_TYPE']
print(principalDf['Economic_Indicator'])

In [ ]:
# Build The Tensor

l = list(principalDf['OCCUPATION_TYPE'])
m = list(principalDf['ID'])
n = list(principalDf['Economic_Indicator'])
print(n)

occupation = dict([(y,x) for x,y in enumerate(set(l))])
ID = dict([(y,x) for x,y in enumerate(set(m))])
eco_in = dict([(int(y),x) for x,y in enumerate(set(n))])



all_values = occupation.values()
occupationNo = max(all_values)

all_values = ID.values()
IDNo = max(all_values)

all_values = eco_in.values()
print(all_values)
eco_inNo = max(all_values)
print(eco_inNo)


indexlist = []
valuelist = []
  

print("Occupationno:",occupationNo)
ex = np.zeros([occupationNo+1,IDNo+1,eco_inNo+1])


for i in range(0,330):
    print(principalDf['Economic_Indicator'][i])
    x = occupation[principalDf['OCCUPATION_TYPE'][i]]
    y = ID[principalDf['ID'][i]]
    z = eco_in[int(principalDf['Economic_Indicator'][i])]

    #print(str(x)+" "+str(y)+" "+str(z))
    info = principalDf['Status'][i]
    
    #print(info)
    print(x,y,z)
    ex[x][y][z] = info


"""for i in range(0,1000):
    a = random.randint(0,31)
    b = random.randint(0,13)
    c = random.randint(0,17)
    if(i == 1):
        print('Co-ordinates:')
        print(a)
        print(b)
        print(c)
        print(d)
        print(e)
        print(f)
    ex[a][b][c] = 1

print(ex)"""


In [ ]:

#initial inputs
s = [2,3,1]
rank = 3


A=ex
factors = non_negative_parafac(A,rank = rank)
print("Weights:",factors[0])
print("Factor 1:",factors[1][0])
print("Factor 2:",factors[1][1])
print("Factor 3:",factors[1][2])



#Selected random indices from mode 1,2 and 3 of initial tensor as the FIXED INDICES
Xs,selectedI,selectedJ,selectedK,selectedindxI,selectedindxJ,selectedindxK = summary(A,s)
print("Xs:",Xs)
print("Selected from mode 1:",selectedindxI)
print("Selected from mode 2:",selectedindxJ)
print("Selected from mode 3:",selectedindxK)



In [ ]:
def SamBATen(X, rank, s, r):
    for i in range(0,r):
        if i==0:
            Xs,selectedI,selectedJ, selectedK, indI,indJ,indK =summary(X,s)
            fixed = []
            fixed.append(selectedI) 
            fixed.append(selectedJ) 
            fixed.append(selectedK)
            print("&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&")
            print("Anew Xs:")
            print("Xs shape: ",Xs.shape)
            print(Xs)
        else:
            Xs,selectedI,selectedJ, selectedK, indI,indJ,indK =summary(X,s,fixed)
            
        factors = non_negative_parafac(Xs,rank = rank, verbose = 0,normalize_factors=False)
        
        weights = factors[0]
        '''
        scaling_matrix = np.array(np.diag(weights**(1/3)))
        As= np.matmul(np.array(factors[1][0]),scaling_matrix)
        Bs= np.matmul(np.array(factors[1][1]),scaling_matrix)
        Cs= np.matmul(np.array(factors[1][2]),scaling_matrix)'''
        As= factors[1][0]
        Bs= factors[1][1]
        Cs= factors[1][2]
        
        print("$$$$$$$$$$$$$$$$$$$$$$$$$$")
        print("Factors")
        print(As)
        print()
        print(Bs)
        print()
        print(Cs)
        print()
        
        if i ==0:
            A, B, C = merge(As,Bs,Cs,i,rank,selectedI,selectedJ, selectedK)
        else :
            A, B, C = merge(As,Bs,Cs,i,rank,selectedI,selectedJ, selectedK,Aold,Bold,Cold)
            
        Aold = A
        Bold = B
        Cold = C
    return A,B,C,weights
            

In [ ]:
def regenerate(factors):
    weights = factors[0]
    a = factors[1][0]
    b = factors[1][1]
    c = factors[1][2]
    print(a.shape)
    print(b.shape)
    print(c.shape)
    temp = np.matmul(a,np.transpose(b))
    #temp2 = numpy.swapaxes(temp, 1, 2)
    print("a * b:",temp)
    print("a*b shape:",temp.shape)
    ##temp2 = np.matmul(temp,c)
    #temp3 = numpy.swapaxes(temp, 1, 2)
    
    return temp
    

In [ ]:
#Add new data repeatedly and apply incremental tensor decompostion for subsequent iterations
additions_no = 1
no_of_new_cols = 1

r = 1
for i in range(0,additions_no):
    F = 10*np.random.rand(A.shape[0],A.shape[1],no_of_new_cols)
    Anew = buildXnew(A,F,selectedI,selectedJ,selectedK)
    print("####################################")
    print("Anew:")
    print(Anew)
    factor1,factor2,factor3,weights = SamBATen(Anew, rank, s, r)
    #print("Components on iteration ",i+1,":")
    #print(factor1,factor2,factor3)
    factors[0] = weights
    factors[1][0] = factor1
    factors[1][1] = factor2
    factors[1][2] = factor3
    factor_array = []
    factor_array.append(factor1)
    factor_array.append(factor2)
    factor_array.append(factor3)
    
    #regenerated_tensor = regenerate(factors)
    regenerated_tensor = tensorly.cp_to_tensor((weights,factor_array))
    
    print("-------------------------------------------------------------------")
    print("Iteratiin no: ",i+1)
    print("New Values:")
    print(F)
    print("Regenerated tensor:")
    print(regenerated_tensor.shape)
    print(regenerated_tensor)




In [ ]:
print(np.linalg.norm(A))
print(np.linalg.norm(regenerated_tensor))
residual=np.linalg.norm(A) - np.linalg.norm(regenerated_tensor)
print(residual)
fit = 1- ((residual) /np.linalg.norm(A))
print("fit",fit)
error= abs(residual) /abs(np.linalg.norm(A))
print("error",error)